In [1]:
from pathlib import Path
from shutil import rmtree

import altair as alt
import kuzu
import polars as pl

from ddl import sct, icd


snomed_data = Path("./data/external/snomed-ips/Snapshot/Terminology/")
kuzu_path = Path("./data/internal/kuzu-db/")
if kuzu_path.exists:
    rmtree(kuzu_path)

db = kuzu.Database(kuzu_path)
conn = kuzu.Connection(db)


def read_snomed(path: Path) -> pl.DataFrame:
    return pl.read_csv(path, separator="\t").with_columns(
        pl.col("effectiveTime").cast(pl.String).str.to_date("%Y%m%d"),
        pl.col("active").cast(pl.Boolean),
    )


In [2]:
Description = read_snomed(
    snomed_data / "sct2_Description_IPSSnapshot-en_IPST_20240701.txt"
)

# Two type of descriptions:
# 900000000000003001 | Fully qualified name
# 900000000000013009 | Synonyms
Description.select(pl.col("typeId").value_counts()).unnest("typeId")

shape: (2, 2)
┌────────────────────┬───────┐
│ typeId             ┆ count │
│ ---                ┆ ---   │
│ i64                ┆ u32   │
╞════════════════════╪═══════╡
│ 900000000000003001 ┆ 19697 │
│ 900000000000013009 ┆ 40417 │
└────────────────────┴───────┘

In [3]:
# load Concept
fullname = Description.filter(pl.col("typeId") == 900000000000003001).select(
    pl.col("conceptId"), pl.col("term").alias("fullQualifiedName")
)
synonyms = (
    Description.filter(pl.col("typeId") == 900000000000013009)
    .select(pl.col("conceptId"), pl.col("term").alias("synonyms"))
    .group_by(pl.col("conceptId"))
    .agg("synonyms")
)

join_concept = dict(how="left", left_on="id", right_on="conceptId")
Concept = (
    read_snomed(snomed_data / "sct2_Concept_IPSSnapshot_IPST_20240701.txt")
    .join(fullname, **join_concept)
    .join(synonyms, **join_concept)
)

conn.execute(sct.concept + "COPY SCT FROM Concept;")

In [4]:
# load Relationship, note we need to change ordering of columns for loading in Kuzu
join_relationship = dict(how="left", left_on="typeId", right_on="conceptId")
Relationship = (
    read_snomed(snomed_data / "sct2_Relationship_IPSSnapshot_IPST_20240701.txt")
    .select(
        pl.col(
            [
                "sourceId",
                "destinationId",
                "id",
                "effectiveTime",
                "active",
                "moduleId",
                "relationshipGroup",
                "typeId",
                "characteristicTypeId",
                "modifierId",
            ]
        )
    )
    .join(fullname, **join_relationship)
    .join(synonyms, **join_relationship)
)

In [5]:
# inspect frequency of each type of relationship out of 66,017 relationships
print(Relationship.shape)

# 116680003 | Is A occurs 32,111 times i.e accounts for half
# 363698007 | Finding site 5,497
# 116676008 | Associated morphology 3,818
type_count = (
    Relationship.select(pl.col("typeId").value_counts())
    .unnest("typeId")
    .sort("count", descending=True)
)
type_count.plot.bar(
    alt.X("count:Q").scale(type="symlog"), y=alt.Y("typeId:O").sort("-x")
)

(66017, 12)


alt.Chart(...)

In [6]:
for name, id in sct.top10_relationships:
    Relationship_ = Relationship.filter(pl.col("typeId") == id)
    conn.execute(
        f"DROP TABLE IF EXISTS {name};"
        + sct.relationship(name)
        + f"COPY {name} FROM Relationship_;"
    )


## ICD-10 2019

ICD-10 structure, from top to bottom:

- Chapter
- Group
- Category
  - three-position, e.g. C88
  - four-position, e.g. C88.9


In [7]:
icd_data = Path("./data/external/icd10-2019/")

ICD10Chapter = pl.read_csv(
    icd_data / "icd102019syst_chapters.txt",
    has_header=False,
    separator=";",
).rename({"column_1": "number", "column_2": "rubric"})

ICD10Group = pl.read_csv(
    icd_data / "icd102019syst_groups.txt", has_header=False, separator=";"
).with_columns(
    pl.concat_str([pl.col("column_1"), pl.col("column_2")], separator="-").alias(
        "code"
    ),
    pl.col("column_3").alias("chapter"),
    pl.col("column_4").alias("rubric"),
)

ICD10Code = pl.read_csv(
    icd_data / "icd102019syst_codes.txt",
    has_header=False,
    separator=";",
    infer_schema_length=10000,
)

Group_to_Chapter = ICD10Group.select("code", "chapter")

Category3_to_group = ICD10Group.join(
    ICD10Code.filter(pl.col("column_1") == 3), left_on="column_1", right_on="column_5"
).select(pl.col("column_7").alias("category3"), pl.col("code").alias("group"))

ICD10Group = ICD10Group.select("code", "rubric")

expr_category = (
    pl.col("column_7").alias("code"),
    pl.col("column_9").alias("rubric"),
)

ICD10Category3 = ICD10Code.filter(pl.col("column_1") == 3).select(expr_category)
ICD10Category4 = ICD10Code.filter(pl.col("column_1") == 4).select(expr_category)

Category4_to_3 = ICD10Category4.select(
    pl.col("code"), pl.col("code").str.head(3).alias("superclass")
)


In [8]:
Group_to_Chapter

shape: (263, 2)
┌─────────┬─────────┐
│ code    ┆ chapter │
│ ---     ┆ ---     │
│ str     ┆ i64     │
╞═════════╪═════════╡
│ A00-A09 ┆ 1       │
│ A15-A19 ┆ 1       │
│ A20-A28 ┆ 1       │
│ A30-A49 ┆ 1       │
│ A50-A64 ┆ 1       │
│ …       ┆ …       │
│ Z55-Z65 ┆ 21      │
│ Z70-Z76 ┆ 21      │
│ Z80-Z99 ┆ 21      │
│ U00-U49 ┆ 22      │
│ U82-U85 ┆ 22      │
└─────────┴─────────┘

In [9]:
for name in ["Chapter", "Group", "Category3", "Category4"]:
    conn.execute(
        f"DROP TABLE IF EXISTS ICD10{name};"
        + icd.__dict__[name]
        + f"COPY ICD10{name} FROM ICD10{name};"
    )

conn.execute(
    "DROP TABLE IF EXISTS IsSubClassOf;"
    + icd.IsSubClassOf
    + "COPY IsSubClassOf_ICD10Category4_ICD10Category3 FROM Category4_to_3;"
    + "COPY IsSubClassOf_ICD10Category3_ICD10Group FROM Category3_to_group;"
    + "COPY IsSubClassOf_ICD10Group_ICD10Chapter FROM Group_to_Chapter;"
)
